In [6]:
import pandas as pd
import os
import FinanceDataReader as fdr
import matplotlib as plt

In [2]:
path = './data'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list.sort_values(by='종목코드',ascending=True)

,종목명,종목코드,상장시장
109,메리츠화재,000060,KOSPI
126,하이트진로,000080,KOSPI
67,유한양행,000100,KOSPI
69,CJ대한통운,000120,KOSPI
156,두산,000150,KOSPI
...,...,...,...
328,넥스틴,348210,KOSDAQ
31,하이브,352820,KOSPI
199,솔브레인,357780,KOSDAQ
176,티와이홀딩스,363280,KOSPI


In [49]:
start_date = '20190101'
end_date = '20201231'
sample_code = '005930'
stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
stock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-01-02,39400,39400,38550,38750,7847664,0.001292
2019-01-03,38300,38550,37450,37600,12471493,-0.029677
2019-01-04,37450,37600,36850,37450,14108958,-0.003989
2019-01-07,38000,38900,37800,38750,12748997,0.034713
2019-01-08,38000,39200,37950,38100,12756554,-0.016774
...,...,...,...,...,...,...
2020-12-23,72400,74000,72300,73900,19411326,0.022130
2020-12-24,74100,78800,74000,77800,32502870,0.052774
2020-12-28,79000,80100,78200,78700,40085044,0.011568


In [50]:
stock.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
Change    0
dtype: int64

In [44]:
print(f'prev : {stock.columns}')
stock.reset_index(inplace=True) 
print(f'post : {stock.columns}')
pd.to_datetime(stock['Date'])

prev : Index(['index', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change'], dtype='object')
post : Index(['level_0', 'index', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Change'],
      dtype='object')


0     2019-01-02
1     2019-01-03
2     2019-01-04
3     2019-01-07
4     2019-01-08
         ...    
489   2020-12-23
490   2020-12-24
491   2020-12-28
492   2020-12-29
493   2020-12-30
Name: Date, Length: 494, dtype: datetime64[ns]

In [51]:
stock.dtypes

Open        int64
High        int64
Low         int64
Close       int64
Volume      int64
Change    float64
dtype: object

In [55]:
# date 살려서 plot 그리기

In [56]:
from sklearn.model_selection import train_test_split
stock_x = stock.drop('Close', axis=1)
stock_y = stock['Close'].copy()
x_train, x_test, y_train, y_test = train_test_split(stock_x, stock_y, test_size=0.33)

In [57]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train) # Date를 reset_index로 분리하면 오류(Type Error) 발생

LinearRegression()

In [58]:
from sklearn.metrics import mean_absolute_error
# TODO : NMAE로 변경하기
y_predict = lin_reg.predict(x_test)
lin_mae = mean_absolute_error(y_test, y_predict)
lin_mae

207.9253152305354